# 25. 데이터 전처리 및 피처 엔지니어링

## 25.1 사용 목적에 따라 모델 서식 지정하기
+ 데이터를 Double Type의 컬럼으로 가져와서 레이블을 표시, Vector 타입의 컬럼을 사용해서 특징을 나타냄
+ 그래프 분석은 정점과 에지가 각각 DataFrame으로 구성되어야 함

In [1]:
# 세션 생성
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("feature engineering examples") \
    .getOrCreate()

spark.conf.set('spark.sql.shuffle.partitions', 5)

In [2]:
sales = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("../BookSamples/data/retail-data/by-day/*.csv")\
    .coalesce(5)\
    .where("Description IS NOT NULL")
    # 널 값을 꼭 걸러내야 함

In [3]:
sales.show(5, False)

+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|580538   |23084    |RABBIT NIGHT LIGHT             |48      |2011-12-05 08:38:00|1.79     |14075.0   |United Kingdom|
|580538   |23077    |DOUGHNUT LIP GLOSS             |20      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22906    |12 MESSAGE CARDS WITH ENVELOPES|24      |2011-12-05 08:38:00|1.65     |14075.0   |United Kingdom|
|580538   |21914    |BLUE HARMONICA IN BOX          |24      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22467    |GUMBALL COAT RACK              |6       |2011-12-05 08:38:00|2.55     |14075.0   |United Kingdom|
+---------+---------+---------------------------

In [4]:
fakeIntDF = spark.read.parquet("../BookSamples/data/simple-ml-integers/")
simpleDF = spark.read.json("../BookSamples/data/simple-ml/")
scaleDF = spark.read.parquet("../BookSamples/data/simple-ml-scaling/")

In [5]:
sales.cache()
sales.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [6]:
fakeIntDF.show()
simpleDF.show()
scaleDF.show()

+----+----+----+
|int1|int2|int3|
+----+----+----+
|   4|   5|   6|
|   7|   8|   9|
|   1|   2|   3|
+----+----+----+

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red|good|    45| 38.97187133755819|
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.

## 25.2 변환자
+ 변환자는 다양한 방식으로 원시 데이터를 변환시키는 함수
+ 새로운 상호작용 변수를 생성하거나 컬럼을 정규화, 모델 입력을 위해 Double형 변환 기능 등을 제공

In [7]:
from pyspark.ml.feature import Tokenizer

tkn = Tokenizer(inputCol="Description", outputCol="words")
tkn.transform(sales.select("Description")).show()

+--------------------+--------------------+
|         Description|               words|
+--------------------+--------------------+
|  RABBIT NIGHT LIGHT|[rabbit, night, l...|
| DOUGHNUT LIP GLOSS |[doughnut, lip, g...|
|12 MESSAGE CARDS ...|[12, message, car...|
|BLUE HARMONICA IN...|[blue, harmonica,...|
|   GUMBALL COAT RACK|[gumball, coat, r...|
|SKULLS  WATER TRA...|[skulls, , water,...|
|FELTCRAFT GIRL AM...|[feltcraft, girl,...|
|CAMOUFLAGE LED TORCH|[camouflage, led,...|
|WHITE SKULL HOT W...|[white, skull, ho...|
|ENGLISH ROSE HOT ...|[english, rose, h...|
|HOT WATER BOTTLE ...|[hot, water, bott...|
|SCOTTIE DOG HOT W...|[scottie, dog, ho...|
|ROSE CARAVAN DOOR...|[rose, caravan, d...|
|GINGHAM HEART  DO...|[gingham, heart, ...|
|STORAGE TIN VINTA...|[storage, tin, vi...|
|SET OF 4 KNICK KN...|[set, of, 4, knic...|
|      POPCORN HOLDER|   [popcorn, holder]|
|GROW A FLYTRAP OR...|[grow, a, flytrap...|
|AIRLINE BAG VINTA...|[airline, bag, vi...|
|AIRLINE BAG VINTA...|[airline, 

## 25.3 전처리 추정자
+ 수행하려는 변환이 입력 컬럼에 대한 데이터 또는 정보로 초기화되어야 할 때 사용
    + StandardScaler는 평균과 분산 파라미터가 필요하므로 이를 추정자가 처리함
+ inputCol, outputCol 지정 필요

In [8]:
from pyspark.ml.feature import StandardScaler

ss = StandardScaler(inputCol="features", outputCol="scaledFeatures")
ss.fit(scaleDF).transform(scaleDF).show()

+---+--------------+--------------------+
| id|      features|      scaledFeatures|
+---+--------------+--------------------+
|  0|[1.0,0.1,-1.0]|[1.19522860933439...|
|  1| [2.0,1.1,1.0]|[2.39045721866878...|
|  0|[1.0,0.1,-1.0]|[1.19522860933439...|
|  1| [2.0,1.1,1.0]|[2.39045721866878...|
|  1|[3.0,10.1,3.0]|[3.58568582800318...|
+---+--------------+--------------------+



## 25.4 고수준 변환자
+ 비즈니스 문제에 집중하기 위해서는 가능한 한 최상위 수준의 변환자 사용을 지향해야 함

### 25.4.1 RFormula
+ R에서 빌려옴
+ 데이터값은 숫자형 또는 범주형이 되고, 문자열에서 값을 추출하는 등의 조직을 할 필요가 없음
+ 기본 연산자
    + ~
        + 함수에서 타깃과 항을 분리
    + \+
        + +0는 y 절편 제거를 의미
    + -
        + 삭제기호, -1는 y 절편 제거를 의미 +0과 결과가 같음
    + :
        + 상호작용(수치형 값이나 이진화된 범주 값에 대한 곱셈)
    + .
        + 타깃 / 종속변수를 제외한 모든 컬럼
        

In [9]:
from pyspark.ml.feature import RFormula

supervise = RFormula(formula="lab ~ . + color:value1 + color:value2")
for item in supervise.fit(simpleDF).transform(simpleDF).toPandas()["features"] :
    print(item)

(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])
(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])
(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])
(10,[1,2,3,5,8],[1.0,15.0,38.97187133755819,15.0,38.97187133755819])
(10,[1,2,3,5,8],[1.0,12.0,14.386294994851129,12.0,14.386294994851129])
(10,[1,2,3,5,8],[1.0,16.0,14.386294994851129,16.0,14.386294994851129])
(10,[0,2,3,4,7],[1.0,35.0,14.386294994851129,35.0,14.386294994851129])
(10,[0,2,3,4,7],[1.0,1.0,38.97187133755819,1.0,38.97187133755819])
(10,[0,2,3,4,7],[1.0,2.0,14.386294994851129,2.0,14.386294994851129])
(10,[0,2,3,4,7],[1.0,16.0,14.386294994851129,16.0,14.386294994851129])
(10,[0,2,3,4,7],[1.0,45.0,38.97187133755819,45.0,38.97187133755819])
(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])
(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])
(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])
(10,[1,2,3,5,8],[1.0,15.0,38.97187133755819,15.0

### 25.4.2 SQL 변환자
+ 테이블 이름 대신 THIS 키워드 사용
+ DataFrame 조작을 전처리 단계로 공식적으로 코딩하거나 하이퍼파이미터 튜닝 시 특징에 서로 다른 SQL 식을 적용하고자 할 때 SQLTransformer를 사용하는 것이 좋음

In [10]:
from pyspark.ml.feature import SQLTransformer

basicTransformation = SQLTransformer()\
    .setStatement("""
        SELECT sum(Quantity), count(*), CustomerID
        FROM __THIS__
        GROUP BY CustomerID
    """)

basicTransformation.transform(sales).show()

+-------------+--------+----------+
|sum(Quantity)|count(1)|CustomerID|
+-------------+--------+----------+
|         1721|     119|   18180.0|
|         1070|     107|   12782.0|
|          701|      59|   17402.0|
|          478|      35|   16642.0|
|          477|      28|   16811.0|
|          986|      71|   15053.0|
|         1419|      50|   12913.0|
|          445|      43|   12628.0|
|         4505|     183|   14401.0|
|          271|      20|   16851.0|
|        63012|    1076|   17511.0|
|         1032|     128|   18044.0|
|         3839|     166|   18198.0|
|         5497|     342|   13001.0|
|         1408|     109|   16379.0|
|         1544|     612|   13230.0|
|         3316|     742|   17757.0|
|        32324|     201|   17404.0|
|         5458|     284|   16705.0|
|         2531|     244|   12957.0|
+-------------+--------+----------+
only showing top 20 rows



### 25.4.3 백터 조합기 
+ 모든 특징을 하나의 큰 벡터로 연결하여 추정자에 전달하는 기능 제공
+ 일반적으로 파이프라인 마지막 단계에서 사용됨
+ 다양한 변환자를 사용하여 여러 가지 조작을 수행하고 그에 대한 모든 결과를 모아야 하는 경우 유용

In [11]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

+----+----+----+------------------------------------+
|int1|int2|int3|VectorAssembler_691a224541e2__output|
+----+----+----+------------------------------------+
|   4|   5|   6|                       [4.0,5.0,6.0]|
|   7|   8|   9|                       [7.0,8.0,9.0]|
|   1|   2|   3|                       [1.0,2.0,3.0]|
+----+----+----+------------------------------------+



## 25.5 연속형 특징 처리하기 
+ 일반적으로 사용되는 2 개의 변환자
    + 버켓팅이라는 프로세스를 통한 연속형 -> 범주형 변환
    + 스케일링 및 정규화

### 25.5.1 버켓팅

+ Bucketizer를 사용하면 주어진 연속형 특징을 지정한 버켓으로 분할함
+ 체중이라는 컬럼을 과체중, 평균, 저체중의 세 가지 버켓으로 나누어 활용할 때 유용
+ 세 가지 요구사항
    + 분할 배열의 최솟값은 DataFrame의 최솟값보다 작음
    + 분할 배열의 최댓값은 DataFrame의 최댓값보다 금
    + 분할 배열은 최소 세 개 이상의 값을 지정해서 두 개 이상의 버켓을 생성

In [12]:
""" 하드 코딩된 값으로 분할 """
from pyspark.ml.feature import Bucketizer

contDF = spark.range(20).selectExpr("cast(id as double)")

bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show()

+----+-------------------------------+
|  id|Bucketizer_c11f751907c9__output|
+----+-------------------------------+
| 0.0|                            0.0|
| 1.0|                            0.0|
| 2.0|                            0.0|
| 3.0|                            0.0|
| 4.0|                            0.0|
| 5.0|                            1.0|
| 6.0|                            1.0|
| 7.0|                            1.0|
| 8.0|                            1.0|
| 9.0|                            1.0|
|10.0|                            2.0|
|11.0|                            2.0|
|12.0|                            2.0|
|13.0|                            2.0|
|14.0|                            2.0|
|15.0|                            2.0|
|16.0|                            2.0|
|17.0|                            2.0|
|18.0|                            2.0|
|19.0|                            2.0|
+----+-------------------------------+



In [13]:
""" 분위수 기준 분할 """
from pyspark.ml.feature import QuantileDiscretizer

# 5등분
bucketer = QuantileDiscretizer().setNumBuckets(5)\
    .setInputCol("id")\
    .setOutputCol("result")
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

+----+------+
|  id|result|
+----+------+
| 0.0|   0.0|
| 1.0|   0.0|
| 2.0|   0.0|
| 3.0|   1.0|
| 4.0|   1.0|
| 5.0|   1.0|
| 6.0|   1.0|
| 7.0|   2.0|
| 8.0|   2.0|
| 9.0|   2.0|
|10.0|   2.0|
|11.0|   2.0|
|12.0|   3.0|
|13.0|   3.0|
|14.0|   3.0|
|15.0|   4.0|
|16.0|   4.0|
|17.0|   4.0|
|18.0|   4.0|
|19.0|   4.0|
+----+------+



+ 이 외 지역성 기반 해싱(locality sensitivityhashing, LSH)과 같은 고급 기법도 제공함

### 25.5.2 스케일링과 정규화
+ MLlib에서는 항상 Vector 타입의 컬럼에서 이 작업이 수행됨
+ 주어진 컬럼의 모든 로우를 조사한 다음 해당 벡터의 모든 차원을 고유한 파라미터로 처리함

#### StandardScaler
+ 평균 0, 표준편차 1인 분포를 갖도록 데이터를 표준화

In [14]:
from pyspark.ml.feature import StandardScaler

sScaler = StandardScaler().setInputCol("features").setOutputCol("scaledFeatures")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+--------------------+
| id|      features|      scaledFeatures|
+---+--------------+--------------------+
|  0|[1.0,0.1,-1.0]|[1.19522860933439...|
|  1| [2.0,1.1,1.0]|[2.39045721866878...|
|  0|[1.0,0.1,-1.0]|[1.19522860933439...|
|  1| [2.0,1.1,1.0]|[2.39045721866878...|
|  1|[3.0,10.1,3.0]|[3.58568582800318...|
+---+--------------+--------------------+



#### MinMaxScaler
+ 모든 값을 0에서 1사이로 변환

In [15]:
from pyspark.ml.feature import MinMaxScaler

minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features").setOutputCol("scaledFeatures")
minMax.fit(scaleDF).transform(scaleDF).show()

+---+--------------+----------------+
| id|      features|  scaledFeatures|
+---+--------------+----------------+
|  0|[1.0,0.1,-1.0]|   [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|   [7.5,5.5,7.5]|
|  0|[1.0,0.1,-1.0]|   [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|   [7.5,5.5,7.5]|
|  1|[3.0,10.1,3.0]|[10.0,10.0,10.0]|
+---+--------------+----------------+



#### MaxAbsScaler
+ 모든 값을 -1에서 1사이로 변환 

In [16]:
from pyspark.ml.feature import MaxAbsScaler

maScaler = MaxAbsScaler().setInputCol("features")
maScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+---------------------------------+
| id|      features|MaxAbsScaler_3e801a03504b__output|
+---+--------------+---------------------------------+
|  0|[1.0,0.1,-1.0]|             [0.33333333333333...|
|  1| [2.0,1.1,1.0]|             [0.66666666666666...|
|  0|[1.0,0.1,-1.0]|             [0.33333333333333...|
|  1| [2.0,1.1,1.0]|             [0.66666666666666...|
|  1|[3.0,10.1,3.0]|                    [1.0,1.0,1.0]|
+---+--------------+---------------------------------+



#### ElementWiseProduct

+ 벡터의 각 값을 임의의 값으로 조정할 수 있을 때 사용

In [17]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors

scaleUpVec = Vectors.dense(10.0, 15.0, 20.0) # 10배, 15배, 20배로 조절
scalingUp = ElementwiseProduct()\
    .setScalingVec(scaleUpVec)\
    .setInputCol("features")\
    .setOutputCol("scaledFeatures")
scalingUp.transform(scaleDF).show()

+---+--------------+-----------------+
| id|      features|   scaledFeatures|
+---+--------------+-----------------+
|  0|[1.0,0.1,-1.0]| [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]| [20.0,16.5,20.0]|
|  0|[1.0,0.1,-1.0]| [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]| [20.0,16.5,20.0]|
|  1|[3.0,10.1,3.0]|[30.0,151.5,60.0]|
+---+--------------+-----------------+



## 25.6  범주형 특징 처리하기

### 25.6.1 StringIndexer

In [18]:
from pyspark.ml.feature import StringIndexer

lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show()
idxRes.printSchema()

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|labelInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|     1.0|
|  red| bad|     1| 38.97187133755819|     0.0|
|  red| bad|     2|14.386294994851129|     0.0|
|  red| bad|    16|14.386294994851129|     0.0|
|  red|good|    45| 38.97187133755819|     1.0|
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|  

In [19]:
""" 숫자형 컬럼은 자동 문자열로 변환되어 처리됨 """

valIndxr = StringIndexer().setInputCol("value1").setOutputCol("valueInd")
valIndxr.fit(simpleDF).transform(simpleDF).show()
valIndxr.fit(simpleDF).transform(simpleDF).printSchema()

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|valueInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     2.0|
| blue| bad|     8|14.386294994851129|     4.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     5.0|
|green|good|    12|14.386294994851129|     0.0|
|green| bad|    16|14.386294994851129|     1.0|
|  red|good|    35|14.386294994851129|     6.0|
|  red| bad|     1| 38.97187133755819|     2.0|
|  red| bad|     2|14.386294994851129|     7.0|
|  red| bad|    16|14.386294994851129|     1.0|
|  red|good|    45| 38.97187133755819|     3.0|
|green|good|     1|14.386294994851129|     2.0|
| blue| bad|     8|14.386294994851129|     4.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     5.0|
|green|good|    12|14.386294994851129|     0.0|
|green| bad|    16|14.386294994851129|     1.0|
|  red|good|    35|14.386294994851129|  

In [20]:
""" 유효하지 않은 값에 대해 오류를 출력하거나 로우를 건너뛰고 처리할 수 있음 """

valIndxr.setHandleInvalid("skip")
valIndxr.fit(simpleDF).setHandleInvalid("skip") ## 

StringIndexer_566dff98bdb2

### 25.6.2 색인된 값을 텍스트로 변환하기
+ 예측 결과(색인)를 원래 범주로 다시 변환하는 데 유용함 

In [21]:
from pyspark.ml.feature import IndexToString

lblReverse = IndexToString().setInputCol("labelInd").setOutputCol("labelReverse")
lblReverse.transform(idxRes).show()

+-----+----+------+------------------+--------+------------+
|color| lab|value1|            value2|labelInd|labelReverse|
+-----+----+------+------------------+--------+------------+
|green|good|     1|14.386294994851129|     1.0|        good|
| blue| bad|     8|14.386294994851129|     0.0|         bad|
| blue| bad|    12|14.386294994851129|     0.0|         bad|
|green|good|    15| 38.97187133755819|     1.0|        good|
|green|good|    12|14.386294994851129|     1.0|        good|
|green| bad|    16|14.386294994851129|     0.0|         bad|
|  red|good|    35|14.386294994851129|     1.0|        good|
|  red| bad|     1| 38.97187133755819|     0.0|         bad|
|  red| bad|     2|14.386294994851129|     0.0|         bad|
|  red| bad|    16|14.386294994851129|     0.0|         bad|
|  red|good|    45| 38.97187133755819|     1.0|        good|
|green|good|     1|14.386294994851129|     1.0|        good|
| blue| bad|     8|14.386294994851129|     0.0|         bad|
| blue| bad|    12|14.38

### 25.6.3 벡터 인덱싱하기

+ 범주형 변수를 대상으로 하는 유용한 도구
+ 벡터 내에 존재하는 범주형 데이터를 자동으로 찾아서 0부터 시작하는 카테고리 색인을 사용하여 범주형 특징으로 변환함
+ 연속형 변수가 반복된 값이 많아서 고유한 값이 너무 적으면 의도하지 않게 범주형 변수로 잘못 변환될 수 있음

In [22]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors

idxIn = spark.createDataFrame([
    (Vectors.dense(1, 2, 3), 1),
    (Vectors.dense(2, 5, 6), 2),
    (Vectors.dense(1, 8, 9), 3)
]).toDF("features", "label")

In [23]:
indxr = VectorIndexer()\
    .setInputCol("features")\
    .setOutputCol("idxed")\
    .setMaxCategories(3)

indxr.fit(idxIn).transform(idxIn).show() # 요소가 3개 미만인 모든 컬럼이 인덱싱 됨

+-------------+-----+-------------+
|     features|label|        idxed|
+-------------+-----+-------------+
|[1.0,2.0,3.0]|    1|[0.0,0.0,0.0]|
|[2.0,5.0,6.0]|    2|[1.0,1.0,1.0]|
|[1.0,8.0,9.0]|    3|[0.0,2.0,2.0]|
+-------------+-----+-------------+



### 25.6.4 원-핫 인코딩

In [24]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

lblIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder().setInputCol("colorInd").setOutputCol("colorOH")
ohe.transform(colorLab).show()

+-----+--------+-------------+
|color|colorInd|      colorOH|
+-----+--------+-------------+
|green|     1.0|(2,[1],[1.0])|
| blue|     2.0|    (2,[],[])|
| blue|     2.0|    (2,[],[])|
|green|     1.0|(2,[1],[1.0])|
|green|     1.0|(2,[1],[1.0])|
|green|     1.0|(2,[1],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|green|     1.0|(2,[1],[1.0])|
| blue|     2.0|    (2,[],[])|
| blue|     2.0|    (2,[],[])|
|green|     1.0|(2,[1],[1.0])|
|green|     1.0|(2,[1],[1.0])|
|green|     1.0|(2,[1],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
|  red|     0.0|(2,[0],[1.0])|
+-----+--------+-------------+
only showing top 20 rows



## 25.7 텍스트 데이터 변환자 

### 25.7.1 텍스트 토큰화하기
+ 자연어를 토큰 또는 개별 단어 목록으로 변환하는 프로세스
+ 가장 쉬운 방법은 Tokenizer 클래스 사용
+ RegexTokenizer를 이용하면 공백뿐만 아니라 정규 표현식을 이용한 Tokenizer를 만들 수 있음

In [25]:
from pyspark.ml.feature import Tokenizer

tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(20, False)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, , water, transfer, tattoos]      |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

In [9]:
""" 정규식 이용 """
from pyspark.ml.feature import RegexTokenizer

rt = RegexTokenizer().setInputCol("Description").setOutputCol("DescOut")\
    .setPattern(" ")\
    .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, water, transfer, tattoos]        |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

In [10]:
""" 사전에 제시된 패턴에 매칭하는 값을 출력(setGaps) """

rt = RegexTokenizer().setInputCol("Description").setOutputCol("DescOut")\
    .setPattern(" ")\
    .setGaps(False)\
    .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------+
|Description                        |DescOut           |
+-----------------------------------+------------------+
|RABBIT NIGHT LIGHT                 |[ ,  ]            |
|DOUGHNUT LIP GLOSS                 |[ ,  ,  ]         |
|12 MESSAGE CARDS WITH ENVELOPES    |[ ,  ,  ,  ]      |
|BLUE HARMONICA IN BOX              |[ ,  ,  ,  ]      |
|GUMBALL COAT RACK                  |[ ,  ]            |
|SKULLS  WATER TRANSFER TATTOOS     |[ ,  ,  ,  ,  ]   |
|FELTCRAFT GIRL AMELIE KIT          |[ ,  ,  ]         |
|CAMOUFLAGE LED TORCH               |[ ,  ]            |
|WHITE SKULL HOT WATER BOTTLE       |[ ,  ,  ,  ,  ]   |
|ENGLISH ROSE HOT WATER BOTTLE      |[ ,  ,  ,  ]      |
|HOT WATER BOTTLE KEEP CALM         |[ ,  ,  ,  ]      |
|SCOTTIE DOG HOT WATER BOTTLE       |[ ,  ,  ,  ]      |
|ROSE CARAVAN DOORSTOP              |[ ,  ]            |
|GINGHAM HEART  DOORSTOP RED        |[ ,  ,  ,  ]      |
|STORAGE TIN VINTAGE LEAF      

### 25.7.2 일반적인 단어 제거하기

+ the, and, but 등 빈번히 발견되는 불용어 제거

In [28]:
from pyspark.ml.feature import StopWordsRemover

englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover()\
    .setStopWords(englishStopWords)\
    .setInputCol("DescOut")\
    .setOutputCol("DescStops")
stops.transform(tokenized).show()

+--------------------+--------------------+--------------------+
|         Description|             DescOut|           DescStops|
+--------------------+--------------------+--------------------+
|  RABBIT NIGHT LIGHT|[rabbit, night, l...|[rabbit, night, l...|
| DOUGHNUT LIP GLOSS |[doughnut, lip, g...|[doughnut, lip, g...|
|12 MESSAGE CARDS ...|[12, message, car...|[12, message, car...|
|BLUE HARMONICA IN...|[blue, harmonica,...|[blue, harmonica,...|
|   GUMBALL COAT RACK|[gumball, coat, r...|[gumball, coat, r...|
|SKULLS  WATER TRA...|[skulls, , water,...|[skulls, , water,...|
|FELTCRAFT GIRL AM...|[feltcraft, girl,...|[feltcraft, girl,...|
|CAMOUFLAGE LED TORCH|[camouflage, led,...|[camouflage, led,...|
|WHITE SKULL HOT W...|[white, skull, ho...|[white, skull, ho...|
|ENGLISH ROSE HOT ...|[english, rose, h...|[english, rose, h...|
|HOT WATER BOTTLE ...|[hot, water, bott...|[hot, water, bott...|
|SCOTTIE DOG HOT W...|[scottie, dog, ho...|[scottie, dog, ho...|
|ROSE CARAVAN DOOR...|[ro

### 25.7.3 단어 조합만들기 (n-gram)



In [29]:
from pyspark.ml.feature import NGram

unigram = NGram().setInputCol("DescOut").setN(1)
bigram = NGram().setInputCol("DescOut").setN(2)
unigram.transform(tokenized.select("DescOut")).show(10, False)
bigram.transform(tokenized.select("DescOut")).show(10, False)

+-------------------------------------+-------------------------------------+
|DescOut                              |NGram_d7807ff4a191__output           |
+-------------------------------------+-------------------------------------+
|[rabbit, night, light]               |[rabbit, night, light]               |
|[doughnut, lip, gloss]               |[doughnut, lip, gloss]               |
|[12, message, cards, with, envelopes]|[12, message, cards, with, envelopes]|
|[blue, harmonica, in, box]           |[blue, harmonica, in, box]           |
|[gumball, coat, rack]                |[gumball, coat, rack]                |
|[skulls, , water, transfer, tattoos] |[skulls, , water, transfer, tattoos] |
|[feltcraft, girl, amelie, kit]       |[feltcraft, girl, amelie, kit]       |
|[camouflage, led, torch]             |[camouflage, led, torch]             |
|[white, skull, hot, water, bottle]   |[white, skull, hot, water, bottle]   |
|[english, rose, hot, water, bottle]  |[english, rose, hot, wate

### 25.7.4 단어를 숫자로 변환하기

+ CountVectorizer 토큰화된 데이터에서 작동함
    + 모든 문서에서 단어 집합을 찾은 다음 문서별로 해당 단어의 출현 빈도를 계산
    + 변환 과정에서 DataFrame 컬럼의 각 로우에서 주어진 단어의 발생 빈도를 계산하고 해당 로우에 포함된 용어를 벡터 형태로 출력함
+ 모든 로우를 문서(document), 모든 단어를 용어(term), 모든 용어의 집합을 어휘집(vocabulary)로 취급

In [30]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer()\
    .setInputCol("DescOut")\
    .setOutputCol("countVec")\
    .setVocabSize(500)\
    .setMinTF(1)\
    .setMinDF(2)

fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show(10, False)

"총 어휘 크기, 어휘에 포함된 단어 색인, 특정 단어의 출현 빈도 순으로 출력됨"

+-------------------------------+-------------------------------------+---------------------------------------------+
|Description                    |DescOut                              |countVec                                     |
+-------------------------------+-------------------------------------+---------------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |(500,[150,185,212],[1.0,1.0,1.0])            |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |(500,[462,463,491],[1.0,1.0,1.0])            |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|(500,[35,41,166],[1.0,1.0,1.0])              |
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |(500,[10,16,36,352],[1.0,1.0,1.0,1.0])       |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |(500,[228,280,407],[1.0,1.0,1.0])            |
|SKULLS  WATER TRANSFER TATTOOS |[skulls, , water, trans

'총 어휘 크기, 어휘에 포함된 단어 색인, 특정 단어의 출현 빈도 순으로 출력됨'

#### TF-IDF

In [31]:
# Red가 포함된 백터
tfIdfIn = tokenized\
    .where("array_contains(DescOut, 'red')")\
    .select("DescOut")\
    .limit(10)
tfIdfIn.show(10, False)

from pyspark.ml.feature import HashingTF, IDF

tf = HashingTF()\
    .setInputCol("DescOut")\
    .setOutputCol("TFOut")\
    .setNumFeatures(10000)

idf = IDF()\
    .setInputCol("TFOut")\
    .setOutputCol("IDFOut")\
    .setMinDocFreq(2)

idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(10, False)

+---------------------------------------+
|DescOut                                |
+---------------------------------------+
|[gingham, heart, , doorstop, red]      |
|[red, floral, feltcraft, shoulder, bag]|
|[alarm, clock, bakelike, red]          |
|[pin, cushion, babushka, red]          |
|[red, retrospot, mini, cases]          |
|[red, kitchen, scales]                 |
|[gingham, heart, , doorstop, red]      |
|[large, red, babushka, notebook]       |
|[red, retrospot, oven, glove]          |
|[red, retrospot, plate]                |
+---------------------------------------+

+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|DescOut                                |TFOut                                                   |IDFOut                                                                                           

### 25.7.5 Word2Vec

In [32]:
from pyspark.ml.feature import Word2Vec

# 각 로우는 문장 또는 문서의 단어 주머니
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# 단어를 백터에 매핑
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print(f"Text: [{', '.join(text)}] => \nVector: {str(vector)}")

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.09164838017895818,-0.03636998124420643,0.0134640232892707]
Text: [I, wish, Java, could, use, case, classes] => 
Vector: [-0.05678207134561879,0.005540126269417149,0.0009586233645677567]
Text: [Logistic, regression, models, are, neat] => 
Vector: [0.07904563546180726,-0.04238809710368514,-0.005285423249006271]


## 25.8 특징 조작하기
+ feature space를 조작하는 알고리즘

### 25.8.1 주성분 분석

In [33]:
from pyspark.ml.feature import PCA

pca = PCA().setInputCol("features").setK(2)
pca.fit(scaleDF).transform(scaleDF).show(20, False)

+---+--------------+------------------------------------------+
|id |features      |PCA_52a9a90b4450__output                  |
+---+--------------+------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|1  |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060149758]|
+---+--------------+------------------------------------------+



### 25.8.2 상호작용
+ Rformual를 사용하여 생성하는 것을 추천

### 25.8.3 다항식 전개
+ 모든 입력 컬럼의 상호작용 변수를 생성하는 데 사용됨
+ 특정 특징 간의 상호작용을 검토해보고는 싶지만 정확히 무엇을 사용해야 할지 확신하지 못할 때 유용하게 사용할 수 있음
+ 특징 공간을 상당히 크게 확장시킬 수 있어서 계산 비용이 많고, 과적합을 초래할 수 있으므로 주의 필요

In [34]:
from pyspark.ml.feature import PolynomialExpansion

pe = PolynomialExpansion().setInputCol("features").setDegree(2)
pe.transform(scaleDF).show()

+---+--------------+----------------------------------------+
| id|      features|PolynomialExpansion_151804874eea__output|
+---+--------------+----------------------------------------+
|  0|[1.0,0.1,-1.0]|                    [1.0,1.0,0.1,0.1,...|
|  1| [2.0,1.1,1.0]|                    [2.0,4.0,1.1,2.2,...|
|  0|[1.0,0.1,-1.0]|                    [1.0,1.0,0.1,0.1,...|
|  1| [2.0,1.1,1.0]|                    [2.0,4.0,1.1,2.2,...|
|  1|[3.0,10.1,3.0]|                    [3.0,9.0,10.1,30....|
+---+--------------+----------------------------------------+



## 25.9 특징 선택

+ 과적합을 방지할 수 있도록 ChiSqSelector와 같은 간단한 옵션을 제공

### 25.9.1 ChiSqSelector

+ 통계적 검정을 활용하여 예측하려는 레이블과 독립적이지 않은 특징을 식별하고 관련 없는 특징을 삭제함
+ 카이제곱 검정을 기반으로 동작함

In [ ]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer

tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn\
    .transform(sales.select("Description", "CustomerId"))\
    .where("CustomerId IS NOT NULL")
prechi = fittedCV.transform(tokenized)\
    .where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
    .setFeaturesCol("countVec")\
    .setLabelCol("CustomerId")\
    .setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi)\
    .drop("customerId", "Description", "DescOut").show()

## 25.10 고급 주제
+ 변환자 저장하기
+ 사용자 정의 변환자 작성하기

### 25.10.1 변환자 저장하기

In [ ]:
""" 저장하기 """
fittedPCA = pca.fit(scaleDF)
fittedPCA.write().overwrite().save("/tmp/fittedPCA")

In [ ]:
from pyspark.ml.feature import PCAModel

""" 불러오기 """
loadedPCA = PCAModel.load("/tmp/fittedPCA")
loadedPCA.transform(scaleDF).show()

### 25.10.2 사용자 정의 변환자 작성하기

+ 일반적으로 내장 모듈은 효율적으로 실행되도록 최적화되어 있으므로 가능한 한 많이 사용
+ 책의 예제는 파이썬 버전이 없으므로 생략